In [326]:
import numpy as np
import pandas as pd

In [327]:
선행지수 = pd.read_csv("./_경기선행지수.csv")
건설업주가지수 = pd.read_csv("./_건설업주가지수.csv")
국고채 = pd.read_csv("./_국고채(3년).csv")
GDP = pd.read_csv("./_명목GDP_실질GDP변화율.csv")
산업생산지수 = pd.read_csv("./_산업생산지수.csv")
소비자물가지수 = pd.read_csv("./_소비자물가지수.csv")
코스피 = pd.read_csv("./_코스피지수.csv")
환율 = pd.read_csv("./_환율.csv")

* 선행지수, 건설업주가지수, 소비자물가지수, 산업생산지수, 환율

In [328]:
선행지수['날짜'] = pd.to_datetime(선행지수['날짜'], format='%Y.%m')
선행지수['날짜'] =  pd.to_datetime(선행지수['날짜'].dt.strftime('%Y-%m'))

In [329]:
건설업주가지수['날짜'] = pd.to_datetime(건설업주가지수['날짜'].apply(lambda x: f"{int(x):04d}-{int((x * 100) % 100):02d}"))

In [330]:
소비자물가지수['날짜'] =  pd.to_datetime(소비자물가지수['날짜'].apply(lambda x: f"{int(x):04d}-{int((x * 100) % 100):02d}"))

In [331]:
산업생산지수['날짜'][산업생산지수['날짜'] == '2023.11 p)'] = '2023.11'
산업생산지수['날짜'][산업생산지수['날짜'] == '2023.12 p)'] = '2023.12'

산업생산지수['날짜'] = pd.to_datetime(산업생산지수['날짜'])
산업생산지수['날짜'] = pd.to_datetime(산업생산지수['날짜'].dt.strftime('%Y-%m'))

<ipython-input-331-58392a65a3d2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  산업생산지수['날짜'][산업생산지수['날짜'] == '2023.11 p)'] = '2023.11'
<ipython-input-331-58392a65a3d2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  산업생산지수['날짜'][산업생산지수['날짜'] == '2023.12 p)'] = '2023.12'


In [332]:
환율['날짜'] = pd.to_datetime(pd.to_datetime(환율['날짜']).dt.strftime('%Y-%m'))


In [430]:
환율 = 환율.rename(columns={'종가' : '환율'})

In [431]:
환율['환율'] = 환율['환율'].str.replace(',', '').astype(float)

* 국고채 전처리

In [333]:
# '월'제거 후 날짜데이터타입으로 변경
국고채['날짜'] = pd.to_datetime(국고채['날짜'].str.replace('월', ''), format='%Y%m')

# 중복행 제거
국고채 = 국고채.drop_duplicates().reset_index(drop=True)
국고채['날짜'] =  pd.to_datetime(국고채['날짜'].dt.strftime('%Y-%m'))

* GDP 전처리

In [334]:
# ','제거 후 수치형으로 변경
GDP['국내총생산(명목GDP)'] = GDP['국내총생산(명목GDP)'].str.replace(',', '').astype(float)

# 분기별 값은 해당 분기에 해당하는 월별 값으로 설정하는 함수 정의
def 분기to월 (row):
    year_분기 = row['날짜']
    year = int(year_분기[:4]) #year만 가져옴
    분기 = int(year_분기[4]) #분기만 가져옴
    months_in_분기 = [(year, 분기 * 3 - 2), (year, 분기 * 3 - 1), (year, 분기 * 3)] #해당 분기에 해당하는 월 list
    return pd.DataFrame({
        '날짜': [f"{year}{month:02d}" for year, month in months_in_분기],
        '국내총생산(명목GDP)': [row['국내총생산(명목GDP)']] * 3,
        '경제성장률(실질GDP성장률)': [row['경제성장률(실질GDP성장률)']] * 3
    })

gdp_new = pd.concat([분기to월(row) for index, row in GDP.iterrows()])

gdp_new.reset_index(drop=True, inplace=True)
gdp_new['날짜'] = pd.to_datetime(gdp_new['날짜'], format='%Y%m')

gdp_new['날짜'] = pd.to_datetime(gdp_new['날짜'].dt.strftime('%Y-%m'))
#gdp_new = gdp_new.set_index('날짜')

* 코스피

In [335]:
코스피['종가'] = 코스피['종가'].str.replace(',', '').astype(float)
코스피['날짜'] = pd.to_datetime(코스피['날짜'])
코스피['날짜'] = pd.to_datetime(코스피['날짜'].dt.strftime('%Y-%m'))
코스피 = 코스피.sort_values('날짜').reset_index(drop = True)

### 새로운 구별 변수 데이터 전처리 시작

In [336]:
아파트거래 = pd.read_csv("./월별_아파트거래.csv")
아파트매매 = pd.read_csv("./월별_아파트매매.csv")
지가변동누적률 = pd.read_csv("./지가변동률_월별.csv")
가계대출 = pd.read_csv("./분기별 가계대출.csv")
물가상승률 = pd.read_csv("./물가상승률.csv")

##### 아파트매매거래 전처리

In [372]:
aprtdf = dict()

def 매매거래전처리(df, a):
    for i in range(1, 217):
        j = 2*i - 1
        df['날짜'][j] = df['날짜'][j-1]


    동호수 = df.iloc[::2].drop(columns = '동|면적', axis = 1)
    면적 = df.iloc[1::2].drop(columns = '동|면적', axis = 1)



    # 올바른 변수명 사용 및 문자열 결합 수정
    aprtdf[f'{a}동호수'] = 동호수.reset_index(drop = True)
    aprtdf[f'{a}면적'] = 면적.reset_index(drop = True)

    def type (df) :
      a = pd.to_datetime(df['날짜'], format='%Y년 %m월')
      df.loc[df.index[0:], '날짜'] = a.dt.strftime('%Y-%m-%d')
      df['날짜'] = pd.to_datetime(df['날짜'])
      return(df)

    aprtdf[f'{a}동호수'] = type(aprtdf[f'{a}동호수'])
    aprtdf[f'{a}면적'] = type(aprtdf[f'{a}면적'])

    return None


In [373]:
매매거래전처리(아파트거래, '거래')
매매거래전처리(아파트매매, '매매')

아파트거래_동호수 = aprtdf['거래동호수']
아파트거래_면적 = aprtdf['거래면적']

아파트매매_동호수 = aprtdf['매매동호수']
아파트매매_면적 = aprtdf['매매면적']

<ipython-input-372-5daf5593c8a7>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['날짜'][j] = df['날짜'][j-1]
<ipython-input-372-5daf5593c8a7>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['날짜'][j] = df['날짜'][j-1]


##### 자가변동률_월별 전처리

In [339]:
for i in range(1, 237):
        j = 2*i - 1
        지가변동누적률['날짜'][j] = 지가변동누적률['날짜'][j-1]

자가변동률_분기 = 지가변동누적률.iloc[::2].drop(columns = '변동률|누계', axis = 1).reset_index(drop = True)
자가누적률_분기 = 지가변동누적률.iloc[1::2].drop(columns = '변동률|누계', axis = 1).reset_index(drop = True)

<ipython-input-339-2995abaf80e4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  지가변동누적률['날짜'][j] = 지가변동누적률['날짜'][j-1]


In [340]:
def 분기를월별로변환(df):
    월별데이터리스트 = []
    for i in range(len(df)):
        year = df['날짜'][i][0:4]  # 연도 추출
        qtr = int(df['날짜'][i][6:7])  # 분기 추출, 정수로 변환

        # 해당 분기에 속하는 각 월의 날짜 생성
        for month in range((qtr-1)*3 + 1, qtr*3 + 1):
            new_row = df.iloc[i].copy()
            new_row['날짜'] = f'{year}-{month:02d}-01'  # YYYY-MM-DD 형식으로 날짜 설정
            월별데이터리스트.append(new_row)

    # pd.concat을 사용하여 리스트에 저장된 모든 데이터 합치기
    월별데이터 = pd.concat(월별데이터리스트, axis=1).transpose().reset_index(drop=True)

    return 월별데이터

자가변동률_분기_앞부분 = 분기를월별로변환(자가변동률_분기[:8])
자가누적률_분기_앞부분 = 분기를월별로변환(자가누적률_분기[:8])

In [341]:
#날짜형 타입설정
def type1 (df) :
      a = pd.to_datetime(df['날짜'], format='%Y년 %m월')
      df.loc[df.index[0:], '날짜'] = a.dt.strftime('%Y-%m-%d')
      return(df)

자가변동률 = pd.concat([자가변동률_분기_앞부분,type1(자가변동률_분기[8:])], axis = 0)
자가누적률 = pd.concat([자가누적률_분기_앞부분,type1(자가누적률_분기[8:])], axis = 0)

<ipython-input-341-3d6a4c789832>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[df.index[0:], '날짜'] = a.dt.strftime('%Y-%m-%d')
<ipython-input-341-3d6a4c789832>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[df.index[0:], '날짜'] = a.dt.strftime('%Y-%m-%d')


In [342]:
# 수치형 타입설정
def type2 (df) :
  df['날짜'] = pd.to_datetime(df['날짜'])

  # '자가변동률_분기'라는 DataFrame을 가정하고, '날짜' 열을 제외한 모든 열을 float으로 변환합니다.
  for column in df.columns:
      if column != '날짜':  # '날짜' 열을 제외
          df[column] = df[column].astype(float)
  return df

In [343]:
자가변동률 = type2(자가변동률)
자가누적률 = type2(자가누적률)

In [404]:
자가변동률

,날짜,서울,종로구,중구,용산구,성동구,광진구,동대문구,중랑구,성북구,...,강서구,구로구,금천구,영등포구,동작구,관악구,서초구,강남구,송파구,강동구
0,2003-01-01,0.340,0.460,0.550,0.190,0.230,0.850,0.390,0.090,0.400,...,0.520,0.590,0.500,0.620,0.570,0.510,0.030,0.230,0.160,0.080
1,2003-02-01,0.340,0.460,0.550,0.190,0.230,0.850,0.390,0.090,0.400,...,0.520,0.590,0.500,0.620,0.570,0.510,0.030,0.230,0.160,0.080
2,2003-03-01,0.340,0.460,0.550,0.190,0.230,0.850,0.390,0.090,0.400,...,0.520,0.590,0.500,0.620,0.570,0.510,0.030,0.230,0.160,0.080
3,2003-04-01,0.480,0.150,0.070,0.280,0.790,1.540,1.050,0.210,0.160,...,0.560,0.560,0.160,0.610,0.140,0.220,0.570,0.710,0.830,0.200
4,2003-05-01,0.480,0.150,0.070,0.280,0.790,1.540,1.050,0.210,0.160,...,0.560,0.560,0.160,0.610,0.140,0.220,0.570,0.710,0.830,0.200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231,2023-08-01,0.144,0.106,0.100,0.181,0.131,0.106,0.145,0.100,0.147,...,0.101,0.099,0.109,0.152,0.104,0.100,0.160,0.263,0.090,0.085
232,2023-09-01,0.205,0.236,0.149,0.179,0.201,0.149,0.245,0.195,0.131,...,0.181,0.164,0.198,0.209,0.125,0.140,0.216,0.333,0.120,0.102
233,2023-10-01,0.251,0.294,0.209,0.254,0.301,0.214,0.311,0.248,0.208,...,0.271,0.239,0.231,0.236,0.205,0.261,0.292,0.317,0.171,0.218
234,2023-11-01,0.227,0.260,0.206,0.277,0.237,0.201,0.233,0.212,0.197,...,0.249,0.183,0.200,0.228,0.200,0.233,0.272,0.261,0.148,0.239


##### 가계대출 전처리

In [344]:
def 분기를월별로변환(df):
    월별데이터리스트 = []
    for i in range(len(df)):
        year = df['날짜'][i][0:4]  # 연도 추출
        qtr = int(df['날짜'][i][4:5])  # 분기 추출, 정수로 변환

        # 해당 분기에 속하는 각 월의 날짜 생성
        for month in range((qtr-1)*3 + 1, qtr*3 + 1):
            new_row = df.iloc[i].copy()
            new_row['날짜'] = f'{year}-{month:02d}-01'  # YYYY-MM-DD 형식으로 날짜 설정
            월별데이터리스트.append(new_row)

    # pd.concat을 사용하여 리스트에 저장된 모든 데이터 합치기
    월별데이터 = pd.concat(월별데이터리스트, axis=1).transpose().reset_index(drop=True)
    월별데이터['날짜'] = pd.to_datetime(월별데이터['날짜'])

    return 월별데이터

가계대출_month = 분기를월별로변환(가계대출)

In [345]:
가계대출_month['가계대출금액'] = 가계대출_month['가계대출금액'].str.replace(',', '').astype(float)

In [405]:
가계대출_month

,날짜,가계대출금액
0,2003-01-01,420.7
1,2003-02-01,420.7
2,2003-03-01,420.7
3,2003-04-01,428.1
4,2003-05-01,428.1
...,...,...
244,2023-05-01,1747.4
245,2023-06-01,1747.4
246,2023-07-01,1759.1
247,2023-08-01,1759.1


#### 물가상승률 전처리

In [349]:
#날짜형 타입설정
def type1 (df) :
      a = pd.to_datetime(df['날짜'], format='%Y%m월')
      df.loc[df.index[0:], '날짜'] = a.dt.strftime('%Y-%m-%d')
      df['날짜'] = pd.to_datetime(df['날짜'])
      return(df)

물가상승률_day = type1(물가상승률)

<ipython-input-349-17e029e0c9ce>:4: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[df.index[0:], '날짜'] = a.dt.strftime('%Y-%m-%d')


In [351]:
# 수치형 타입설정
def type2 (df) :
  for column in df.columns:
      if column != '날짜':  # '날짜' 열을 제외
          df[column] = df[column].astype(float)
  return df

물가상승률 = type2(물가상승률_day)

### 구별 데이터 [dict형태]

##### 구별로 나눠져 있는 변수들을 구별_변수 데이터로 만들기

In [451]:
gu = [ '서울', '종로구', '중구', '용산구', '성동구', '광진구', '동대문구', '중랑구', '성북구',
       '강북구', '도봉구', '노원구', '은평구', '서대문구', '마포구', '양천구', '강서구', '구로구', '금천구',
       '영등포구', '동작구', '관악구', '서초구', '강남구', '송파구', '강동구']

In [458]:
#1 날짜, 구_변수를 컬럼으로 하는 데이터프레임 26(구+서울) * 6개(변수)를 딕셔너리형태로 저장함
gu_dict = {}
def gu_df(df,name):
  for i in gu :
    gu_dict[f'{i}_{name}'] = pd.DataFrame(df[['날짜', i]]).rename(columns = {i : i + '_' + name})

gu_df(아파트매매_면적, '매매면적')
gu_df(아파트매매_동호수, '매매동호수')
gu_df(아파트거래_면적, '거래면적')
gu_df(아파트거래_동호수, '거래동호수')
gu_df(자가변동률, '자가변동률')
gu_df(자가누적률, '자가누적률')

In [462]:
#2 같은 구끼리 변수들을 합침 -> 서울+구별 데이터프레임 26개를 딕셔너리형태로 저장함.
gu_var = {}
vars = ['자가변동률','자가누적률','매매면적','매매동호수','거래면적','거래동호수']
for i in gu:
    df_var = [gu_dict[f'{i}_{j}'] for j in vars]  # 각 구별로, 모든 변수에 대한 데이터 프레임 리스트 생성
    gu_var[f'{i}_변수'] = reduce(lambda left, right: pd.merge(left, right, on='날짜', how='outer'), df_var)

##### 구 상관없는 공통 변수들을 합쳐서 데이터프레임 형성

In [465]:
from functools import reduce
var = [선행지수, 건설업주가지수, 소비자물가지수,산업생산지수, 환율,
국고채, gdp_new,코스피, 가계대출_month,물가상승률]
def merge_dataframes_by_date(dfs):
    # reduce 함수를 사용하여 리스트 내의 모든 데이터프레임을 순차적으로 합칩니다.
    # pd.merge 함수의 on='날짜' 옵션으로 '날짜' 열을 기준으로 합치고,
    # how='outer' 옵션으로 모든 날짜를 포함시킵니다.
    merged_v = reduce(lambda left, right: pd.merge(left, right, on='날짜', how='outer'), dfs)
    return merged_v

# var 리스트에 포함된 모든 데이터프레임을 합칩니다.
general_data = merge_dataframes_by_date(var)

##### 구별로 나눠져 있는 데이터프레임[구_변수]과 공통 데이터프레임을 합침 -> 총 서울+구 26개 데이터 나옴 [dict형태]

In [471]:
final_gu_vars = {}
for i in gu:
    # 각 구별로 병합된 데이터 프레임을 생성하고 final_gu_vars 딕셔너리에 저장
    final_gu_vars[f'{i}_df'] = pd.merge(gu_var[f'{i}_변수'], general_data, on='날짜', how='left')

final_gu_vars['구이름_df']

#### 최종 구별 변수 데이터프레임을 item값으로 갖는 딕셔너리 final_gu_vars를 pkl로 저장

In [478]:
import pickle

with open('./final_gu_vars.pkl', 'wb') as f:
    pickle.dump(final_gu_vars, f)

열리는지 확인

In [476]:
with open('./final_gu_vars.pkl', 'rb') as f:
    최종딕셔너리 = pickle.load(f)